<a href="https://colab.research.google.com/github/Nk173/Gulden_2013/blob/vectorized_2/ray_gulden_vectorised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
#import model
import numpy as np
import pandas as pd
from agents_vec import gulden_vectorised

#import tuning library
# !pip install ray
# !pip install -U "ray[default]"
# !pip install bayesian-optimization
# !pip install optuna
import bayes_opt
import optuna

import ray
from ray import air, tune, train
from ray.air import session
from hyperopt import hp
from ray.tune.search.hyperopt import HyperOptSearch

from ray.tune.search.bayesopt import BayesOptSearch

from ray.tune.search.optuna import OptunaSearch
from ray import air


class Tuner:
    def __init__(self, parameters):
        self.parameters=parameters



    def tune_model(self):
      try:
        parameters=self.parameters
        space = {}
        for i in range(parameters["n_countries"]):
            for j in range(parameters["n_products"]):
                space["alpha-"+str(i)+"-"+str(j)]=tune.uniform(0.01, 0.99)
                space["A-"+str(i)+"-"+str(j)]=tune.uniform(0.1, 5)





        algo = BayesOptSearch(random_search_steps=4)
        # algo = OptunaSearch()

        self.tuner = tune.Tuner(
            tune.with_resources( tune.with_parameters(run_sim,parameters=parameters), resources={"CPU": 2}),
            tune_config=tune.TuneConfig(
                metric="error_sum",
                mode="min",
                search_alg=algo,
                num_samples=10000,
            ),
            run_config=train.RunConfig(stop={"training_iteration": 200}, failure_config=train.FailureConfig(max_failures=5)),
            param_space=space,
        )
        self.results = self.tuner.fit()
        print(self.results)
        return self.results
      except Exception as e:
        print(f"Error during tuning: {e}")
        return None

def run_sim(config,parameters):
    try:
      A_test=np.ones((n_countries, n_products))
      alpha_test = np.ones((n_countries, n_products))
      beta_test = np.ones((n_countries, n_products))
      for i in range(parameters["n_countries"]):
          for j in range(parameters["n_products"]):
              alpha_test[i][j]=config["alpha-"+str(i)+"-"+str(j)]
              beta_test[i][j]=1-config["alpha-"+str(i)+"-"+str(j)]
              A_test[i][j]=config["A-"+str(i)+"-"+str(j)]


      production, income, trades_df =  gulden_vectorised(case="tune_session", n_countries=parameters["n_countries"], n_products=parameters["n_products"], countries=parameters["countries"], products=parameters["products"], citizens_per_nation=parameters["citizens_per_nation"], A=A_test, alpha=alpha_test, beta=beta_test,
                        iterations=parameters["iterations"], Tr_time=parameters["Tr_time"], trade_change=parameters["trade_change"], autarky_time=parameters["autarky_time"], pricing_algorithm=parameters["pricing_algorithm"],
                        utility_algorithm=parameters["utility_algorithm"], wage_algorithm=parameters["wage_algorithm"],
                        share=parameters["share"],
                        csv=False, plot=False, shock=parameters["shock"], shock_time=parameters["shock_time"], cm_time=parameters["cm_time"], d=parameters["d"],
                        innovation=parameters["innovation"], innovation_time=parameters["innovation_time"], gamma=parameters["gamma"], eta=parameters["eta"], weights=parameters["weights"], elasticities=parameters["elasticities"],
                        sigma=parameters["sigma"])


      v2=trades_df.values
      v1=parameters["benchmark"].values
      error=np.sum((v1/np.sum(v1)-v2/np.sum(v2))/(v1/np.sum(v1)+v2/np.sum(v2)))**2

      # error=np.sum(((trades_df.values/np.sum(trades_df.values)) - (parameters["benchmark"]/np.sum(parameters["benchmark"])))**2)
      print(error)
      session.report(
          {
              "error_sum": error,
          }

      )
      return {
              "error_sum": error,
          }

    except Exception as e:

        print(f"Error occurred with parameters {config}: {e}")
        # Optionally, log the error to a file or take other actions

        # Return a default error value or handle it as you see fit
        return {
            "error_sum": np.inf  # or some other default/error value
        }



In [ ]:
n_countries=5
n_products=5
# ray.init()
benchmark_matrix = pd.read_csv('IO_cc_5x5.csv', index_col=[0,1])

v=Tuner(parameters={
            "benchmark":benchmark_matrix,
            "n_countries":5,
            "n_products":5,
            "countries" : ['{i}'.format(i=i) for i in range(n_countries)],
            "products" : ['{i}'.format(i=i) for i in range(n_products)],
            "citizens_per_nation" : [100]*n_countries,
            "iterations":2000,
            "Tr_time":1,
            "trade_change":1,
            "autarky_time":15000,
            "pricing_algorithm":'cpmu',
                      "utility_algorithm":'geometric',
            "wage_algorithm":'marginal_product',
                      "share" : np.ones(n_countries),
            "shock":None,
            "shock_time":10000,
            "cm_time":1,
            "d":0.000,
                      "innovation":False,
            "innovation_time":10000,
            "gamma":1,
            "eta":0.01,
            "weights":None,
            "elasticities":None,
                      "sigma":None
        })

#
results = v.tune_model()
if results is not None:
    # Process results
    results=v.tuner.get_results()

else:
    print("Tuning did not complete successfully.")

2024-01-03 09:24:51,802	INFO tune.py:592 -- [output] This will use the new output engine with verbosity 1. To disable the new output and use the legacy output engine, set the environment variable RAY_AIR_NEW_OUTPUT=0. For more information, please see https://github.com/ray-project/ray/issues/36949
2024-01-03 09:24:51,825	WARNING callback.py:137 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


+----------------------------------------------------------------+
| Configuration for experiment     run_sim_2024-01-03_09-24-51   |
+----------------------------------------------------------------+
| Search algorithm                 SearchGenerator               |
| Scheduler                        FIFOScheduler                 |
| Number of trials                 10000                         |
+----------------------------------------------------------------+

View detailed results here: /root/ray_results/run_sim_2024-01-03_09-24-51

Trial status: 1 PENDING
Current time: 2024-01-03 09:24:51. Total running time: 0s
Logical resource usage: 0/2 CPUs, 0/0 GPUs
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

 72%|███████▏  | 1445/2000 [00:25<00:13, 39.91it/s]



Trial status: 1 RUNNING | 1 PENDING
Current time: 2024-01-03 09:25:22. Total running time: 30s
Logical resource usage: 2.0/2 CPUs, 0/0 GPUs
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| Trial name         status       alpha-0-0     A-0-0     alpha-0-1     A-0-1     alpha-0-2     A-0-2     alpha-0-3     A-0-3     alpha-0-4      A-0-4     alpha-1-0      A-1-0     alpha-1-1      A-1-1     alpha-1-2   

100%|█████████▉| 1994/2000 [00:33<00:00, 68.54it/s]


...

Trial status: 37 TERMINATED | 1 RUNNING | 1 PENDING
Current time: 2024-01-03 09:50:39. Total running time: 25min 48s
Logical resource usage: 2.0/2 CPUs, 0/0 GPUs
Current best trial: 9608d006 with error_sum=1101.283945233063 and params={'alpha-0-0': 0.6584424604879148, 'A-0-0': 3.1253503747323803, 'alpha-0-1': 0.18440737026154308, 'A-0-1': 1.7275927853327278, 'alpha-0-2': 0.16098578820114115, 'A-0-2': 4.275961503914036, 'alpha-0-3': 0.673847292699585, 'A-0-3': 2.1494931250899905, 'alpha-0-4': 0.704638224256844, 'A-0-4': 3.724125830156324, 'alpha-1-0': 0.6958298099144913, 'A-1-0': 4.076748164923334, 'alpha-1-1': 0.723482754351805, 'A-1-1': 2.9605356923486728, 'alpha-1-2': 0.99, 'A-1-2': 2.478860288531885, 'alpha-1-3': 0.99, 'A-1-3': 0.6702809412047613, 'alpha-1-4': 0.37137246752852116, 'A-1-4': 0.1, 'alpha-2-0': 0.34476537795473683, 'A-2-0': 5.0, 'alpha-2-1': 0.3082496037553447, 'A-2-1': 0.927089328137334, 'alpha-2-2': 0.2872187169523196, 'A-2-2': 2.609353071306253, 'alpha-2-3': 0.248631

100%|██████████| 2000/2000 [00:44<00:00, 45.16it/s]


(run_sim pid=11064) 1392.592680338434

Trial run_sim_a64ae0a4 completed after 2 iterations at 2024-01-03 09:50:40. Total running time: 25min 48s
+-------------------------------------------+
| Trial run_sim_a64ae0a4 result             |
+-------------------------------------------+
| checkpoint_dir_name                       |
| time_this_iter_s                  0.00038 |
| time_total_s                      44.3469 |
| training_iteration                      2 |
| error_sum                         1392.59 |
+-------------------------------------------+

Trial run_sim_19f62ddd started with configuration:
+-------------------------------------------+
| Trial run_sim_19f62ddd config             |
+-------------------------------------------+
| A-0-0                             3.41311 |
| A-0-1                              1.8476 |
| A-0-2                             4.47957 |
| A-0-3                             2.58487 |
| A-0-4                             3.39993 |
| A-1-0              

 53%|█████▎    | 1062/2000 [00:29<00:23, 40.13it/s]



Trial status: 38 TERMINATED | 1 RUNNING | 1 PENDING
Current time: 2024-01-03 09:51:09. Total running time: 26min 18s
Logical resource usage: 2.0/2 CPUs, 0/0 GPUs
Current best trial: 9608d006 with error_sum=1101.283945233063 and params={'alpha-0-0': 0.6584424604879148, 'A-0-0': 3.1253503747323803, 'alpha-0-1': 0.18440737026154308, 'A-0-1': 1.7275927853327278, 'alpha-0-2': 0.16098578820114115, 'A-0-2': 4.275961503914036, 'alpha-0-3': 0.673847292699585, 'A-0-3': 2.1494931250899905, 'alpha-0-4': 0.704638224256844, 'A-0-4': 3.724125830156324, 'alpha-1-0': 0.6958298099144913, 'A-1-0': 4.076748164923334, 'alpha-1-1': 0.723482754351805, 'A-1-1': 2.9605356923486728, 'alpha-1-2': 0.99, 'A-1-2': 2.478860288531885, 'alpha-1-3': 0.99, 'A-1-3': 0.6702809412047613, 'alpha-1-4': 0.37137246752852116, 'A-1-4': 0.1, 'alpha-2-0': 0.34476537795473683, 'A-2-0': 5.0, 'alpha-2-1': 0.3082496037553447, 'A-2-1': 0.927089328137334, 'alpha-2-2': 0.2872187169523196, 'A-2-2': 2.609353071306253, 'alpha-2-3': 0.24863

In [ ]:
!ray stop
